<a href="https://colab.research.google.com/github/Dhawaldwivedi/Bca-5th-sem-AI-mmdu/blob/main/ai-ml2-g1/sentimentals_detection_using_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np

In [11]:
import zipfile
import os

# Specify the path to your zip file and the directory to extract to
zip_file_path = 'sentimental.zip'  # Updated zip file name
extracted_dir = 'extracted_dataset'

# Create the extraction directory if it doesn't exist
os.makedirs(extracted_dir, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract specific files
    zip_ref.extract('reviews.txt', extracted_dir)
    zip_ref.extract('labels.txt', extracted_dir)


print(f"Specified files extracted to: {extracted_dir}")

Specified files extracted to: extracted_dataset


In [15]:
# Assuming the dataset is a CSV file named 'your_dataset.csv' inside the extracted directory
# Replace 'your_dataset.csv' with the actual filename of your dataset
reviews_file_path = os.path.join(extracted_dir, 'reviews.txt')
labels_file_path = os.path.join(extracted_dir, 'labels.txt')


# Load the datasets into pandas DataFrames (assuming they are text files, reading as lines)
try:
    with open(reviews_file_path, 'r') as f:
        reviews = f.readlines()
    with open(labels_file_path, 'r') as f:
        labels = f.readlines()

    # You might want to further process these lists into DataFrames or other structures
    print("Reviews and labels loaded successfully!")
    print("First 5 reviews:", reviews[:5])
    print("First 5 labels:", labels[:5])

except FileNotFoundError:
    print(f"Error: Dataset files not found at {reviews_file_path} or {labels_file_path}")
except Exception as e:
    print(f"An error occurred while loading the dataset: {e}")

Reviews and labels loaded successfully!
First 5 reviews: ['bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   \n', 'story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrifi

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

training_size = 20000

training_sentences = reviews[0:training_size]
testing_sentences = reviews[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

vocab_size = 10000
max_length = 512
padding_type = 'post'
trunc_type = 'post'

tokenizer = Tokenizer(num_words=vocab_size, oov_token = "<OOV>")
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Convert labels to numerical format (0 for negative, 1 for positive) and remove newline characters
training_labels_numpy = np.array([1 if 'positive' in label else 0 for label in training_labels])
testing_labels_numpy = np.array([1 if 'positive' in label else 0 for label in testing_labels])

training_padded_numpy = np.array(training_padded)
testing_padded_numpy = np.array(testing_padded)

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [14]:
%pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 746.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 137.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 127.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.0 MB/s eta 0:00:00


In [17]:
import tensorflow as tf

multiLayerModel = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

multiLayerModel.build(input_shape=(None, max_length)) # Explicitly build the model

multiLayerModel.summary()

multiLayerModel.compile(loss='binary_crossentropy',
                        optimizer='adam',
                        metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 512, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 512, 128)       │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 64)             │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 751,489 (2.87 MB)

 Trainable params: 751,489 (2.87 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
history = multiLayerModel.fit(
            training_padded_numpy,
            training_labels_numpy,
            validation_data=(testing_padded_numpy, testing_labels_numpy),
            epochs=15,
            verbose=1)

Epoch 1/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 203s 319ms/step - accuracy: 0.6632 - loss: 0.5794 - val_accuracy: 0.8290 - val_loss: 0.4153
Epoch 2/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 199s 319ms/step - accuracy: 0.8781 - loss: 0.3203 - val_accuracy: 0.7972 - val_loss: 0.4419
Epoch 3/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 199s 319ms/step - accuracy: 0.8952 - loss: 0.2763 - val_accuracy: 0.8284 - val_loss: 0.4463
Epoch 4/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 200s 320ms/step - accuracy: 0.9365 - loss: 0.1764 - val_accuracy: 0.8164 - val_loss: 0.4139
Epoch 5/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 198s 318ms/step - accuracy: 0.9447 - loss: 0.1519 - val_accuracy: 0.7976 - val_loss: 0.5160
Epoch 6/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 199s 319ms/step - accuracy: 0.9343 - loss: 0.1852 - val_accuracy: 0.7904 - val_loss: 0.5258
Epoch 7/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 199s 318ms/step - accuracy: 0.8833 - loss: 0.2972 - val_accuracy: 0.8374 - val_loss: 0.4433
Epoch 8/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 199s 319ms/step - accuracy: 0.9282 -

In [19]:
sentences = ['This movie was absolutely fantastic!',
             'I was very disappointed with the service.']

sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
predictions = multiLayerModel.predict(padded)

for i in range(len(predictions)):
    if predictions[i] > 0.5: # Corrected the condition
        print('POSITIVE')
    else:
        print('NEGATIVE')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step
POSITIVE
NEGATIVE


In [20]:
sentences = ['Simply awful and disappointed! It’s a very shallow story, and there in no time to feel an affinity for the main character/characters. It’s almost like whoever edited this movie, edited the first 30% and the last 30% and the movie begins as if you missed part of it. It ends, as if the signal on your TV went out, and it’s abrupt snd makes no sense! OMG! Wait for this movie to come out on DVD! HONESTLY! I am an avid movie goer!',
             'Great movie overall! Hoping for a 2nd one to pickup where the ending left off. Left with a cliffhanger so fingers crossed.']

sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
predictions = multiLayerModel.predict(padded)

for i in range(len(predictions)):
    if predictions[i] > 0.5: # Corrected the condition
        print('POSITIVE')
    else:
        print('NEGATIVE')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
NEGATIVE
POSITIVE
